In [6]:
import numpy as np
import os
from tensorflow.keras.models import model_from_json
from pystoi.stoi import stoi
import mir_eval #https://github.com/craffel/mir_eval
import librosa
import tensorflow as tf
# import keras
from scipy.io import wavfile
from pypesq import pesq #https://github.com/vBaiCai/python-pesq
import vqmetrics
from natsort import natsorted
from tensorflow.keras.models import load_model
from sklearn import preprocessing

In [7]:
#define reconstruct function to reconstruct sound from framed signal.
def reconstruct(wave,angle):
    recon = np.sqrt(np.power(10, wave))
    recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=256, win_length=512, window='hann')
    return recon

In [8]:
loaded_model = load_model(os.path.normpath(os.path.join(Data_path,'partly_trained.h5')))

In [9]:
# # from keras.models import load_model
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras import layers

# w=3
# h = [1024,512]
# len_data = (684108, 257)
# model = Sequential()
# act=layers.LeakyReLU(alpha=0.1)
# model.add(Dense(h[0], input_dim = w*len_data[1]))
# # model.add(BatchNormalization())
# model.add(act)
# act=layers.LeakyReLU(alpha=0.1)
# model.add(Dense(h[1]))
# model.add(act)
# act=layers.LeakyReLU(alpha=0.1)
# model.add(Dense(len_data[1]))


In [10]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
# model_path = os.path.normpath(os.path.join(Data_path,'checkpoints', 'normal','cp-0050.ckpt'))
# model.load_weights(model_path)

In [ ]:
# from keras.models import load_model

# Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
# # json_file = open(os.path.normpath(os.path.join(Data_path,'models','model_3h_dataset.json')), 'r')
# model_path = os.path.normpath(os.path.join(Data_path,'models','model_3h_dataset.json'))
# # loaded_model_json = json_file.read()
# # json_file.close()
# # loaded_model = model_from_json(loaded_model_json)
# loaded_model = load_model(model_path)

In [11]:
import tensorflow as tf
import os
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
json_file = open(os.path.normpath(os.path.join(Data_path,'models','model_3h_dataset_3.json')), 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(os.path.normpath(os.path.join(Data_path,'models','model_3h_dataset_3.h5')))
print("Loaded model from disk")

Loaded model from disk


In [15]:
stoi_eval=[]
sdr=[]
stoi_mixed=[]
sdr_mixed=[]
mean_sdr=[]
mean_sdr_mixed=[]
mean_stoi=[]
mean_stoi_mixed=[]
# orig_path=os.getcwd()
Data_path = 'D:/studies/university/thesis/speech_separation_codes/du16/donesomestuff'
w = 3
inputs = os.listdir(os.path.join(Data_path,'ftr_refrmd_test'))
inputs = natsorted(inputs)
mixed = os.listdir(os.path.join(Data_path,'test_log2'))
mixed = natsorted(mixed)
clean1 = os.listdir(os.path.join(Data_path,'test_log','clean'))
clean1 = natsorted(clean1)
clean = clean1.copy()
j=0
parent = 'results'
foldername='results_3h_3'
print(len(clean))
for i in range(len(inputs)-1):
    clean=np.concatenate((clean,clean1),axis=0)
phase = os.listdir(os.path.join(Data_path,'test_phase2'))
phase = natsorted(phase)
if not os.path.exists(os.path.join(Data_path,parent,foldername)):
    os.makedirs(os.path.join(Data_path,parent,foldername))
if not os.path.exists(os.path.join(Data_path,parent,foldername,'pesq')):
#     print('creating pesq')
    os.makedirs(os.path.join(Data_path,parent,foldername,'pesq'))
if not os.path.exists(os.path.join(Data_path,parent,foldername,'sdr')):
    os.makedirs(os.path.join(Data_path,parent,foldername,'sdr'))
if not os.path.exists(os.path.join(Data_path,parent,foldername,'stoi')):
    os.makedirs(os.path.join(Data_path,parent,foldername,'stoi'))
for filename1,filename2,filename4 in zip(inputs,phase,mixed):
    print(filename1)
    print(filename2)
    print(filename4)
    a = os.listdir(os.path.join(Data_path,'ftr_refrmd_test',filename1))
    b = os.listdir(os.path.join(Data_path,'test_phase2',filename2))
    print('blah')
    c = os.listdir(os.path.join(Data_path,'test_log2',filename4))
    stoi_eval=[]
    pesq_eval=[]
    sdr=[]
    stoi_mixed=[]
    sdr_mixed=[]
    if not os.path.exists(os.path.join(Data_path,parent,foldername,filename1)):
        os.mkdir(os.path.join(Data_path,parent,foldername,filename1))
    for filename11,filename22,filename33,filename44 in zip(a,b,clean,c):
        X_log=np.loadtxt(os.path.join(Data_path,'ftr_refrmd_test',filename1,filename11),delimiter=',')
        print(X_log.shape)
        X_log = preprocessing.scale(X_log, axis=1, with_mean=True, with_std=True, copy=True)
        print(np.std(X_log))
        X_phase=np.loadtxt(os.path.join(Data_path,'test_phase2',filename2,filename22),delimiter=',')
        target = np.loadtxt(os.path.join(Data_path,'test_log','clean',filename33),delimiter=',')
        framed_data=librosa.core.stft(target, n_fft=512, hop_length=256, win_length=512, window='hann')
        abslt=np.absolute(framed_data)**2
        dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
        mixed_files=np.loadtxt(os.path.join(Data_path,'test_log2',filename4,filename44),delimiter=',')
        prediction = loaded_model.predict(X_log)
        recon_out = reconstruct(prediction, X_phase)
        recon_mixed = reconstruct(mixed_files, X_phase)
        recon_clean = reconstruct(dft_signal.T, X_phase)
        wavfile.write(os.path.join(Data_path,parent,foldername,filename1)+'\\'+filename11.replace('.txt','')+'.wav',16000,recon_out)
        target = target[0:len(recon_out)]
        #target = np.reshape(target,(1,len(recon_out)))
        #recon_out=np.reshape(recon_out,(1,len(recon_out)))
#         pesq_eval.append(get_pesq(target, recon_out,16000))
        sdr.append(mir_eval.separation.bss_eval_sources(target, recon_out, compute_permutation=False)[0][0])
        stoi_eval.append(stoi(target, recon_out, 16000, extended=False))
        sdr_mixed.append(mir_eval.separation.bss_eval_sources(target, recon_mixed, compute_permutation=False)[0][0])
        stoi_mixed.append(stoi(target, recon_mixed, 16000, extended=False))
    mean_stoi.append(np.mean(stoi_eval))
    mean_stoi_mixed.append(np.mean(stoi_mixed))
    mean_sdr.append(np.mean(sdr))
    mean_sdr_mixed.append(np.mean(sdr_mixed))
np.savetxt(os.path.join(Data_path,parent,foldername,'sdr')+'\\'+'predictedsdr.txt',mean_sdr, fmt='%1.4f')
np.savetxt(os.path.join(Data_path,parent,foldername,'sdr')+'\\'+'mixedsdr.txt',mean_sdr_mixed, fmt='%1.4f')
np.savetxt(os.path.join(Data_path,parent,foldername,'stoi')+'\\'+'predictedstoi.txt',mean_stoi, fmt='%1.4f')
np.savetxt(os.path.join(Data_path,parent,foldername,'stoi')+'\\'+'mixedstoi.txt',mean_stoi_mixed, fmt='%1.4f')
        

20
0
0
0
blah
(122, 771)
0.9999999999999999
(111, 771)
1.0
(114, 771)
1.0
(101, 771)
0.9999999999999999
(116, 771)
1.0
(116, 771)
0.9999999999999999
(116, 771)
1.0
(114, 771)
1.0
(121, 771)
1.0
(114, 771)
1.0
(121, 771)
1.0
(111, 771)
1.0
(108, 771)
1.0
(120, 771)
1.0
(116, 771)
1.0
(110, 771)
1.0
(111, 771)
1.0
(112, 771)
1.0
(118, 771)
1.0
(110, 771)
1.0
3
3
3
blah
(122, 771)
0.9999999999999999
(111, 771)
1.0
(114, 771)
1.0
(101, 771)
1.0
(116, 771)
1.0
(116, 771)
0.9999999999999999
(116, 771)
1.0
(114, 771)
1.0
(121, 771)
1.0
(114, 771)
1.0
(121, 771)
0.9999999999999999
(111, 771)
1.0000000000000002
(108, 771)
1.0
(120, 771)
0.9999999999999999
(116, 771)
1.0
(110, 771)
1.0
(111, 771)
1.0
(112, 771)
1.0
(118, 771)
1.0
(110, 771)
0.9999999999999999
6
6
6
blah
(122, 771)
1.0
(111, 771)
0.9999999999999999
(114, 771)
1.0
(101, 771)
1.0
(116, 771)
1.0
(116, 771)
1.0
(116, 771)
1.0
(114, 771)
1.0
(121, 771)
1.0
(114, 771)
1.0
(121, 771)
1.0
(111, 771)
1.0
(108, 771)
1.0


KeyboardInterrupt: 

In [14]:
mean_sdr

[0.6693284360807059,
 1.8824042421698857,
 2.8409742174070693,
 -0.7172409709172369,
 -2.3017126997260933,
 -4.025477839106158]

In [15]:
mean_sdr_mixed

[0.3102860913406525,
 3.2391316120756075,
 6.200104618041733,
 -2.5593159169272797,
 -5.322344893230758,
 -7.901038729183284]

In [13]:
def get_pesq(reference, degraded, sample_rate=None, program='pesq'):
    """ Return PESQ quality estimation (two values: PESQ MOS and MOS LQO) based
    on reference and degraded speech samples comparison.
    Sample rate must be 8000 or 16000 (or can be defined reading reference file
    header).
    PESQ utility must be installed.
    """
    if not os.path.isfile(reference) or not os.path.isfile(degraded):
        raise ValueError('reference or degraded file does not exist')
    if not sample_rate:
        import wave
        w = wave.open(reference, 'r')
        sample_rate = w.getframerate()
        w.close()
    if sample_rate not in (8000, 16000):
        raise ValueError('sample rate must be 8000 or 16000')
    import subprocess
    args = [ program, '+%d' % sample_rate, reference, degraded  ]
    pipe = subprocess.Popen(args, stdout=subprocess.PIPE)
    out, _ = pipe.communicate()
    last_line = out.split('\n')[-2]
    if not last_line.startswith('P.862 Prediction'):
        raise ValueError(last_line)
    return tuple(map(float, last_line.split()[-2:]))

In [ ]:
os.path.isfile('down_mixed501_1_16bit.wav')

In [ ]:
np.reshape(target,(1,target.shape[0])).shape

In [ ]:
b

In [ ]:
mean_sdr_mixed

In [ ]:
mean_sdr

In [ ]:
mean_stoi_mixed

In [ ]:
mean_stoi

In [ ]:
import sounddevice as sd
sd.play(recon_out,16000)

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
loaded_model.get_weights()[7].shape

In [ ]:
sdr_mixed

In [ ]:
target[1000]

In [ ]:
filename2

In [ ]:
framed_data=librosa.core.stft(target, n_fft=512, hop_length=256, win_length=512, window='hann')
abslt=np.absolute(framed_data)**2
dft_signal=np.log10(abslt+1e-7*np.ones(np.shape(abslt)))
reconstruct(dft_signal.T,X_phase).shape

In [ ]:
filename4

In [ ]:
target.shape

In [ ]:
import sounddevice as sd
# recon = reconstruct(X_log[:,0:257], X_phase)
samplerate=16000
sd.play(recon_clean, samplerate)
#sd.play(target.T, samplerate)


In [ ]:
recon_mixed.shape

In [ ]:
orig_path=os.getcwd()
w = 7
inputs = os.listdir(os.path.join(orig_path,'ftr_refrmd_test'))
inputs = natsorted(inputs)
clean1 = os.listdir(os.path.join(orig_path,'test_log','clean'))
clean1 = natsorted(clean1)
clean = clean1.copy()
print(len(clean))
for i in range(len(inputs)-1):
    clean=np.concatenate((clean,clean1),axis=0)

In [ ]:
len(clean)


In [ ]:
X_log=np.loadtxt('test_log\\'+'3'+'\\'+'t2_bwbn2s_m2_brif5p'+'.txt', delimiter=',')
X_phase=np.loadtxt('test_phase\\'+'3'+'\\'+'t2_bwbn2s_m2_brif5p'+'.txt', delimiter=',')
recon = reconstruct(X_log, X_phase)
sdr=mir_eval.separation.bss_eval_sources(recon, recon, compute_permutation=True)[0][0]
sir=mir_eval.separation.bss_eval_sources(recon, recon, compute_permutation=True)[1][0]
sar=mir_eval.separation.bss_eval_sources(recon, recon, compute_permutation=True)[2][0]
stoi_ev=stoi(recon, recon, 16000, extended=False)


In [ ]:
pesq_ev=vqmetrics.pesq('t2_bwigzs_m2_sbit3p.wav', 't2_bwigzs_m2_sbit3p.wav',16000)

In [ ]:
pesq_ev=pesq(recon, recon, 16000)

In [ ]:
import subprocess
process = subprocess.Popen(command, stdout=tempFile, shell=True)